In [ ]:
import numpy as np
import tqdm
import json

In [ ]:
file = "./17.txt"
lines = open(file, "r").readlines()
lines = [l.strip("\n") for l in lines]

registerA = 50230824
registerB = 0
registerC = 0

program = [
    2,
    4,  # change B to A%8
    1,
    3,  # change B to B xor 3
    7,
    5,  # change C to A/32
    0,
    3,  # change A to int(A/8)
    1,
    4,  # change B to B xor 4
    4,
    7,  # change B to B xor C
    5,
    5,  # outputs B%8
    3,
    0,  # start from 0
]

In [ ]:
def xor(n1, n2):
    return n1 ^ n2


def run_operation(instr, op, pointer, registerA, registerB, registerC, outs):
    lit_op = op
    if op < 7:
        combo_op = op if op <= 3 else [registerA, registerB, registerC][op - 4]
    else:
        combo_op = None
    if instr == 0:
        res = registerA / (2**combo_op)
        registerA = int(res)
        pointer += 2
    elif instr == 1:
        res = xor(registerB, lit_op)
        registerB = res
        pointer += 2
    elif instr == 2:
        res = combo_op % 8
        registerB = res
        pointer += 2
    elif instr == 3:
        if registerA != 0:
            pointer = lit_op
        else:
            pointer *= 2
    elif instr == 4:
        res = xor(registerB, registerC)
        registerB = res
        pointer += 2
    elif instr == 5:
        out = combo_op % 8
        outs.append(out)
        pointer += 2
    elif instr == 6:
        res = registerA / (2**combo_op)
        registerB = int(res)
        pointer += 2
    elif instr == 7:
        res = registerA / (2**combo_op)
        registerC = int(res)
        pointer += 2
    return pointer, registerA, registerB, registerC, outs

In [ ]:
registerA = 50230824
initial_registerB = 0
initial_registerC = 0

program = [2, 4, 1, 3, 7, 5, 0, 3, 1, 4, 4, 7, 5, 5, 3, 0]
pointer = 0
outs = []
n = 0

for initial_registerA in tqdm.tqdm(range(10000)):
    wrong = False
    registerA = initial_registerA
    registerB = initial_registerB
    registerC = initial_registerC
    pointer = 0
    outs = []
    n = 0
    while (
        pointer < len(program) - 1
        and len(outs) <= len(program)
        and n < 10000
        and not wrong
    ):
        instr, op = program[pointer], program[pointer + 1]
        pointer, registerA, registerB, registerC, outs = run_operation(
            instr, op, pointer, registerA, registerB, registerC, outs
        )
        n += 1
        if program[: len(outs)] != outs[: len(outs)]:
            # print(f'for registerA={initial_registerA} out is alredy wrong, {outs}!={program}')
            wrong = True

    if [int(i) for i in outs] == [int(i) for i in program]:
        print(f"Found register A={initial_registerA} -> {outs}=={program}")

In [ ]:
initial_registerB = 0
initial_registerC = 0

program = [2, 4, 1, 3, 7, 5, 0, 3, 1, 4, 4, 7, 5, 5, 3, 0]

pows_res = {i: [] for i in range(20)}
for pow in range(20):
    regArange = (8**pow, 8 ** (pow + 1))
    for _ in range(1000):
        initial_registerA = np.random.randint(regArange[0], regArange[1])
        registerA = initial_registerA
        registerB = initial_registerB
        registerC = initial_registerC
        pointer = 0
        outs = []
        n = 0

        while pointer < len(program) - 1 and n < 10000:
            instr, op = program[pointer], program[pointer + 1]
            pointer, registerA, registerB, registerC, outs = run_operation(
                instr, op, pointer, registerA, registerB, registerC, outs
            )
            n += 1
        # print(f'initial_registerA: {initial_registerA}, pow: {pow}, len outs {len(outs)}, outs = {outs}')
        pows_res[pow].append(len(outs))
        assert len(outs) == pow + 1, f"found strange case : {len(outs)}, {pow}"
# print(json.dumps(pows_res,indent=2))

In [ ]:
print([8**i for i in range(10)])

In [ ]:
initial_registerB = 0
initial_registerC = 0

program = [
    2,
    4,  # change B to A%8
    1,
    3,  # change B to B xor 3
    7,
    5,  # change C to A/32
    0,
    3,  # change A to int(A/8)
    1,
    4,  # change B to B xor 4
    4,
    7,  # change B to B xor C
    5,
    5,  # outputs B%8
    3,
    0,  # start from 0
]
len_program = len(program)
feasible_range = (8**len_program, 8 ** (len_program + 1))


for D in range(180, 195):
    # A = int((feasible_range[1]-feasible_range[0])/2)+D
    A = feasible_range[0] + D
    registerA = A
    registerB = initial_registerB
    registerC = initial_registerC
    pointer = 0
    outs = []
    n = 0
    print(f"STARTING WITH REGISTER A = {A}")
    while pointer < len(program) - 1 and len(outs) <= len(program) and n < 10000:
        instr, op = program[pointer], program[pointer + 1]
        pointer, registerA, registerB, registerC, outs = run_operation(
            instr, op, pointer, registerA, registerB, registerC, outs
        )
        # print(f'performed op {(instr,op)}, pointer:{pointer}, registerA:{registerA}, registerB:{registerB}, registerC:{registerC}, outs:{outs}')
    print(f"with D={D}, outs: {outs}")

In [ ]:
initial_registerB = 0
initial_registerC = 0

program = [
    2,4, #change B to A%8
    1,3, #change B to B xor 3
    7,5, #change C to A/32
    0,3, #change A to int(A/8)
    1,4, #change B to B xor 4
    4,7, #change B to B xor C
    5,5, #outputs B%8
    3,0 #start from 0
    ]
len_program = len(program)
feasible_range = (8**(len_program-1), 8**(len_program))

found = False
currentA = feasible_range[0]
num_tentative = 0
print(f'PROGRAM : {program}')
while not found and num_tentative<1000:

    registerA=currentA
    registerB=initial_registerB
    registerC=initial_registerC
    pointer = 0
    outs = []
    n = 0
    print(f'STARTING WITH REGISTER A = {currentA}, {currentA-feasible_range[0]} above range start')
    while pointer<len(program)-1 and len(outs)<=len(program) and n<10000:
        instr, op = program[pointer],program[pointer+1]
        pointer, registerA, registerB, registerC, outs = run_operation(instr, op, pointer, registerA, registerB, registerC, outs)
        # print(f'performed op {(instr,op)}, pointer:{pointer}, registerA:{registerA}, registerB:{registerB}, registerC:{registerC}, outs:{outs}')
    found = True
    print(f'outs: {outs}')
    for i in range(len(program)-1, -1, -1):
        if outs[i]!=program[i]:
            print(f'found biggest difference al element {i}, adding 8^{i}={8**i}')
            currentA+=(8**i)
            found=False
            break
        num_tentative += 1

print(f'PROGRAM : {program}')



In [ ]:
# initial_registerB = 0
# initial_registerC = 0

# program = [2, 4, 1, 3, 7, 5, 0, 3, 1, 4, 4, 7, 5, 5, 3, 0]
# pointer = 0
# outs = []
# nn = 0

# len_program = len(program)
# feasible_range = (8**len_program, 8 ** (len_program + 1))
# print(f"feasible range is {feasible_range}, dim: {feasible_range[1]-feasible_range[0]}")
# for initial_registerA in tqdm.tqdm(range(feasible_range[0], feasible_range[1])):
#     wrong = False
#     registerA = initial_registerA
#     registerB = initial_registerB
#     registerC = initial_registerC
#     pointer = 0
#     outs = []
#     n = 0
#     while (
#         pointer < len(program) - 1
#         and len(outs) <= len(program)
#         and n < 10000
#         and not wrong
#     ):
#         instr, op = program[pointer], program[pointer + 1]
#         pointer, registerA, registerB, registerC, outs = run_operation(
#             instr, op, pointer, registerA, registerB, registerC, outs
#         )
#         n += 1
#         if program[: len(outs)] != outs[: len(outs)]:
#             # print(f'for registerA={initial_registerA} out is alredy wrong, {outs}!={program}')
#             wrong = True

#     if [int(i) for i in outs] == [int(i) for i in program]:
#         print(f"Found register A={initial_registerA} -> {outs}=={program}")
        